In [1]:
#
# LSTM모델을 이용해서 주가예측
#
import os
import settings
import pandas as pd        # 라이브러리
import numpy as np         # 라이브러리
from keras.models import Sequential      # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import Dense           # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import LSTM            # 딥러닝을 구동하는 데 필요한 케라스 함수
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
from sklearn.metrics import mean_squared_error
import pymysql         # 파이썬에서 mysql연동시켜주는 라이브러리

#
# DB테이블 값 조회 (SELECT)
#
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')

# 오차율 가져오기
try:
    with connection.cursor() as cursor:
        sql = "select accuracy from stock_hye WHERE company_name='두산'"
        cursor.execute("set names utf8")
        cursor.execute(sql)
        result_accuracy = cursor.fetchone()
        
        for i in result_accuracy:
            accuracy = i
            
finally:
    connection.close()

    
# 데이터셋 생성 함수
look_back = 1
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
 
#
# 저장되어있는 주식데이터 불러오기
#
sydtpath = os.path.join(settings.BASE_DIR, 'chart_data/%s' % (settings.get_today_str()))
stock_code = "dusan"
fullpath = sydtpath + os.path.sep + stock_code + '.csv'
pandf = pd.read_csv(fullpath, index_col="Date")

# 데이터 전처리
nparr = pandf['Close'].values[1:]     # 맨처음 'Close'데이터부터 차례대로 nparr에 저장
print(nparr)
nparr.astype('float32')    # float형으로 변환
print(nparr)
nparr = nparr.reshape(-1,1)
print(nparr)
 
# 정규화 (0~1사이의 값으로 바꿔준다)
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
 
# 학습용, 테스트용 데이터로 나누기 (90%를 학습용 데이터, 10%를 테스트용 데이터)
train_size = int(len(nptf) * 0.9)
test_size = len(nptf) - train_size
train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
print(len(train), len(test))
 
# 학습을 위한 데이터셋 생성하기 (학습용, 테스트용으로 구분)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
 
# RNN모델은 3차원 데이터
# trainX, testX값을 [samples, time steps, features] 형태로 reshape
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
 
# LSTM모델
model = Sequential()                                                # 딥러닝 구조, 층을 설정
model.add(LSTM(20, input_shape=(1, 1)))     # (timestep, feature)   # 층이 추가됨 (add)   # 입력층, 첫번째 은닉층                   
model.add(Dense(1))                                                 # 출력층 (하나)
# 모델을 컴파일 (컴퓨터가 알아들을 수 있도록)   # 오차함수, 최적화 방법
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])   # metrics : 모델 수행 결과를 나타내게 설정 (과적합 문제 방지)
model.fit(trainX, trainY, epochs=1000, batch_size=50, verbose=2)      # 모델을 실제로 수행     # batch_size : 전체 데이터를 10개씩 사용
                                                                                               # verbose(로깅)  2 : epoch당 나오게

# 예측값 평가하기 (얼마나 정확한지)
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)                 # testPredict : 예측 값
testY = scaler.inverse_transform(testY)                             # testY : 실제 값
testScore = math.sqrt(mean_squared_error(testY, testPredict))       # mean_squared_error : 평균 제곱근 오차
print('Train Score: %.2f RMSE' % testScore)                         # 예측 값과 실제 값 차이 출력
 
# 예측 데이터 출력
lastX = nptf[-1]
lastX = np.reshape(lastX, (1, 1, 1))
lastY = model.predict(lastX)
predict = scaler.inverse_transform(lastY)                    # 정규화 시킨 값을 역변환
print('Predict the Close value of final day: %d' % predict)  # 데이터 입력 마지막 다음날 종가 예측
    
# 차트출력, 저장
font_path = "C:/Windows/Fonts/a옛날목욕탕L.ttf"
fontprop = fm.FontProperties(fname=font_path, size=15)

plt.plot(testPredict)
plt.plot(testY)

plt.title('[두산] 예측 그래프\n예측값 : %d \t\t\t\t\t\t\t\t오차율 : %s' % (predict, accuracy), fontproperties=fontprop)

#plt.savefig("./chart_picture/dusan.png",dpi=300)
plt.savefig("C:\source\SPF\chart_picture\dusan.png", dpi=300)
#plt.show()

# 사진 데이터 binary형식으로 바꿔주는 함수
def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData


# 
# DB테이블 값 삽입 (INSERT)
#
# MySQL Connection 연결
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')
try:
    with connection.cursor() as cursor:
        sql = 'INSERT INTO stock_hye (company_name, stock_price, image) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE stock_price = VALUES(stock_price), image = VALUES(image)'
        image = convertToBinaryData("C:\source\SPF\chart_picture\dusan.png")
        cursor.execute(sql, ('두산', int(predict), image))          # 넣으려는 값
    connection.commit()
    
finally:
    connection.close()

Using TensorFlow backend.


[ 93375  93273 104600 ... 101000 100500  97600]
[ 93375  93273 104600 ... 101000 100500  97600]
[[ 93375]
 [ 93273]
 [104600]
 ...
 [101000]
 [100500]
 [ 97600]]
2308 257


C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/1000
 - 3s - loss: 0.2050 - acc: 4.3365e-04
Epoch 2/1000
 - 0s - loss: 0.0896 - acc: 4.3365e-04
Epoch 3/1000
 - 1s - loss: 0.0296 - acc: 8.6730e-04
Epoch 4/1000
 - 0s - loss: 0.0162 - acc: 0.0013
Epoch 5/1000
 - 1s - loss: 0.0145 - acc: 0.0013
Epoch 6/1000
 - 0s - loss: 0.0132 - acc: 0.0013
Epoch 7/1000
 - 1s - loss: 0.0119 - acc: 0.0013
Epoch 8/1000
 - 0s - loss: 0.0106 - acc: 0.0013
Epoch 9/1000
 - 0s - loss: 0.0093 - acc: 0.0013
Epoch 10/1000
 - 0s - loss: 0.0081 - acc: 0.0013
Epoch 11/1000
 - 0s - loss: 0.0069 - acc: 0.0013
Epoch 12/1000
 - 0s - loss: 0.0057 - acc: 0.0013
Epoch 13/1000
 - 1s - loss: 0.0047 - acc: 0.0013
Epoch 14/1000
 - 1s - loss: 0.0038 - acc: 0.0013
Epoch 15/1000
 - 1s - loss: 0.0030 - acc: 0.0013
Epoch 16/1000
 - 0s - loss: 0.0023 - acc: 0.0013
Epoch 17/1000
 - 0s - loss: 0.0018 - acc: 0.0013
Epoch 18/1000
 - 1s - loss: 0.0014 - acc: 0.0013
Epoch 19/1000
 - 1s - loss: 0.0012 - acc: 0.0013
Epoch 20/1000
 - 1s - loss: 0.0010 - acc: 0.0013
Epoch 21/1000
 - 

Epoch 156/1000
 - 0s - loss: 7.5104e-04 - acc: 0.0013
Epoch 157/1000
 - 0s - loss: 7.3122e-04 - acc: 0.0013
Epoch 158/1000
 - 0s - loss: 7.3050e-04 - acc: 0.0013
Epoch 159/1000
 - 0s - loss: 7.3144e-04 - acc: 0.0013
Epoch 160/1000
 - 0s - loss: 7.2480e-04 - acc: 0.0013
Epoch 161/1000
 - 0s - loss: 7.2883e-04 - acc: 0.0013
Epoch 162/1000
 - 1s - loss: 7.2829e-04 - acc: 0.0013
Epoch 163/1000
 - 0s - loss: 7.3081e-04 - acc: 0.0013
Epoch 164/1000
 - 1s - loss: 7.2747e-04 - acc: 0.0013
Epoch 165/1000
 - 0s - loss: 7.3877e-04 - acc: 0.0013
Epoch 166/1000
 - 0s - loss: 7.2955e-04 - acc: 0.0013
Epoch 167/1000
 - 0s - loss: 7.3693e-04 - acc: 0.0013
Epoch 168/1000
 - 0s - loss: 7.3861e-04 - acc: 0.0013
Epoch 169/1000
 - 0s - loss: 7.3276e-04 - acc: 0.0013
Epoch 170/1000
 - 0s - loss: 7.3340e-04 - acc: 0.0013
Epoch 171/1000
 - 0s - loss: 7.2838e-04 - acc: 0.0013
Epoch 172/1000
 - 0s - loss: 7.3491e-04 - acc: 0.0013
Epoch 173/1000
 - 0s - loss: 7.2171e-04 - acc: 0.0013
Epoch 174/1000
 - 0s - loss:

Epoch 308/1000
 - 0s - loss: 7.2652e-04 - acc: 0.0013
Epoch 309/1000
 - 0s - loss: 7.2640e-04 - acc: 0.0013
Epoch 310/1000
 - 0s - loss: 7.2832e-04 - acc: 0.0013
Epoch 311/1000
 - 0s - loss: 7.2655e-04 - acc: 0.0013
Epoch 312/1000
 - 0s - loss: 7.3774e-04 - acc: 0.0013
Epoch 313/1000
 - 0s - loss: 7.3724e-04 - acc: 0.0013
Epoch 314/1000
 - 0s - loss: 7.2647e-04 - acc: 0.0013
Epoch 315/1000
 - 0s - loss: 7.3430e-04 - acc: 0.0013
Epoch 316/1000
 - 0s - loss: 7.5349e-04 - acc: 0.0013
Epoch 317/1000
 - 0s - loss: 7.3384e-04 - acc: 0.0013
Epoch 318/1000
 - 0s - loss: 7.2764e-04 - acc: 0.0013
Epoch 319/1000
 - 0s - loss: 7.3308e-04 - acc: 0.0013
Epoch 320/1000
 - 0s - loss: 7.3251e-04 - acc: 0.0013
Epoch 321/1000
 - 0s - loss: 7.2731e-04 - acc: 0.0013
Epoch 322/1000
 - 0s - loss: 7.2405e-04 - acc: 0.0013
Epoch 323/1000
 - 0s - loss: 7.2817e-04 - acc: 0.0013
Epoch 324/1000
 - 0s - loss: 7.3768e-04 - acc: 0.0013
Epoch 325/1000
 - 0s - loss: 7.2242e-04 - acc: 0.0013
Epoch 326/1000
 - 0s - loss:

Epoch 460/1000
 - 0s - loss: 7.3124e-04 - acc: 0.0013
Epoch 461/1000
 - 0s - loss: 7.5048e-04 - acc: 0.0013
Epoch 462/1000
 - 0s - loss: 7.4080e-04 - acc: 0.0013
Epoch 463/1000
 - 0s - loss: 7.2953e-04 - acc: 0.0013
Epoch 464/1000
 - 0s - loss: 7.2976e-04 - acc: 0.0013
Epoch 465/1000
 - 0s - loss: 7.2149e-04 - acc: 0.0013
Epoch 466/1000
 - 0s - loss: 7.2767e-04 - acc: 0.0013
Epoch 467/1000
 - 0s - loss: 7.3031e-04 - acc: 0.0013
Epoch 468/1000
 - 0s - loss: 7.2929e-04 - acc: 0.0013
Epoch 469/1000
 - 0s - loss: 7.5957e-04 - acc: 0.0013
Epoch 470/1000
 - 0s - loss: 7.2950e-04 - acc: 0.0013
Epoch 471/1000
 - 0s - loss: 7.2567e-04 - acc: 0.0013
Epoch 472/1000
 - 0s - loss: 7.3967e-04 - acc: 0.0013
Epoch 473/1000
 - 0s - loss: 7.2137e-04 - acc: 0.0013
Epoch 474/1000
 - 0s - loss: 7.3049e-04 - acc: 0.0013
Epoch 475/1000
 - 0s - loss: 7.3504e-04 - acc: 0.0013
Epoch 476/1000
 - 0s - loss: 7.3056e-04 - acc: 0.0013
Epoch 477/1000
 - 0s - loss: 7.3354e-04 - acc: 0.0013
Epoch 478/1000
 - 0s - loss:

Epoch 612/1000
 - 0s - loss: 7.2910e-04 - acc: 0.0013
Epoch 613/1000
 - 0s - loss: 7.2845e-04 - acc: 0.0013
Epoch 614/1000
 - 0s - loss: 7.3087e-04 - acc: 0.0013
Epoch 615/1000
 - 0s - loss: 7.2847e-04 - acc: 0.0013
Epoch 616/1000
 - 0s - loss: 7.3122e-04 - acc: 0.0013
Epoch 617/1000
 - 0s - loss: 7.2662e-04 - acc: 0.0013
Epoch 618/1000
 - 0s - loss: 7.3319e-04 - acc: 0.0013
Epoch 619/1000
 - 0s - loss: 7.3244e-04 - acc: 0.0013
Epoch 620/1000
 - 0s - loss: 7.3041e-04 - acc: 0.0013
Epoch 621/1000
 - 0s - loss: 7.4711e-04 - acc: 0.0013
Epoch 622/1000
 - 0s - loss: 7.3273e-04 - acc: 0.0013
Epoch 623/1000
 - 0s - loss: 7.2466e-04 - acc: 0.0013
Epoch 624/1000
 - 0s - loss: 7.3381e-04 - acc: 0.0013
Epoch 625/1000
 - 1s - loss: 7.2671e-04 - acc: 0.0013
Epoch 626/1000
 - 0s - loss: 7.2973e-04 - acc: 0.0013
Epoch 627/1000
 - 0s - loss: 7.2501e-04 - acc: 0.0013
Epoch 628/1000
 - 0s - loss: 7.2498e-04 - acc: 0.0013
Epoch 629/1000
 - 0s - loss: 7.2699e-04 - acc: 0.0013
Epoch 630/1000
 - 0s - loss:

Epoch 764/1000
 - 0s - loss: 7.3396e-04 - acc: 0.0013
Epoch 765/1000
 - 0s - loss: 7.4536e-04 - acc: 0.0013
Epoch 766/1000
 - 0s - loss: 7.3208e-04 - acc: 0.0013
Epoch 767/1000
 - 0s - loss: 7.2852e-04 - acc: 0.0013
Epoch 768/1000
 - 0s - loss: 7.4846e-04 - acc: 0.0013
Epoch 769/1000
 - 0s - loss: 7.3749e-04 - acc: 0.0013
Epoch 770/1000
 - 0s - loss: 7.2646e-04 - acc: 0.0013
Epoch 771/1000
 - 0s - loss: 7.3462e-04 - acc: 0.0013
Epoch 772/1000
 - 0s - loss: 7.3951e-04 - acc: 0.0013
Epoch 773/1000
 - 0s - loss: 7.2997e-04 - acc: 0.0013
Epoch 774/1000
 - 0s - loss: 7.3152e-04 - acc: 0.0013
Epoch 775/1000
 - 0s - loss: 7.4976e-04 - acc: 0.0013
Epoch 776/1000
 - 0s - loss: 7.2860e-04 - acc: 0.0013
Epoch 777/1000
 - 0s - loss: 7.2709e-04 - acc: 0.0013
Epoch 778/1000
 - 0s - loss: 7.3139e-04 - acc: 0.0013
Epoch 779/1000
 - 0s - loss: 7.2558e-04 - acc: 0.0013
Epoch 780/1000
 - 0s - loss: 7.3258e-04 - acc: 0.0013
Epoch 781/1000
 - 0s - loss: 7.3506e-04 - acc: 0.0013
Epoch 782/1000
 - 0s - loss:

Epoch 916/1000
 - 0s - loss: 7.2961e-04 - acc: 0.0013
Epoch 917/1000
 - 0s - loss: 7.3187e-04 - acc: 0.0013
Epoch 918/1000
 - 0s - loss: 7.3000e-04 - acc: 0.0013
Epoch 919/1000
 - 0s - loss: 7.2424e-04 - acc: 0.0013
Epoch 920/1000
 - 0s - loss: 7.2488e-04 - acc: 0.0013
Epoch 921/1000
 - 0s - loss: 7.3684e-04 - acc: 0.0013
Epoch 922/1000
 - 0s - loss: 7.3283e-04 - acc: 0.0013
Epoch 923/1000
 - 0s - loss: 7.2922e-04 - acc: 0.0013
Epoch 924/1000
 - 0s - loss: 7.2533e-04 - acc: 0.0013
Epoch 925/1000
 - 0s - loss: 7.2430e-04 - acc: 0.0013
Epoch 926/1000
 - 0s - loss: 7.3166e-04 - acc: 0.0013
Epoch 927/1000
 - 0s - loss: 7.2344e-04 - acc: 0.0013
Epoch 928/1000
 - 0s - loss: 7.2970e-04 - acc: 0.0013
Epoch 929/1000
 - 0s - loss: 7.3450e-04 - acc: 0.0013
Epoch 930/1000
 - 0s - loss: 7.3864e-04 - acc: 0.0013
Epoch 931/1000
 - 0s - loss: 7.3136e-04 - acc: 0.0013
Epoch 932/1000
 - 1s - loss: 7.2894e-04 - acc: 0.0013
Epoch 933/1000
 - 0s - loss: 7.4968e-04 - acc: 0.0013
Epoch 934/1000
 - 0s - loss: